In [45]:
import pandas as pd
import re
pd.options.display.max_columns = 30  #Can't have too many columns.
import glob
from datetime import datetime
import dateutil.parser
import nltk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']= (17,5)
from collections import Counter
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer
from sklearn.cluster import KMeans
import langid
from tqdm import tqdm
import os
import locale 

In [57]:
#Checking if all the speeches are loaded or not
folder = nltk.data.find('A:/Data Analysis Jupyter/Campaign-Analytics/Speeches/narendramodi/2017/')
path_link = 'A:/Data Analysis Jupyter/Campaign-Analytics/Speeches/narendramodi/2017/*'
paths = glob.glob(path_link)
len(paths)

92

In [58]:
nm_speeches = [ ]  #Empty dataframe to store all the content of speeches
for path in paths:
    corpusReader = nltk.corpus.PlaintextCorpusReader(folder, '.*\.txt')
    # opening the files and converting it to a dictionary
    with open(path, encoding="utf-8") as speech_file:
        filename = path.split('/')[-1]
        date = re.search(r"(_\d\d)_(\w+)_(\d\d\d\d)", filename) 
        if date:
            date = datetime.strptime(date.group(0), '_%d_%b_%Y')
        else:
            date = np.nan
        speech = {
            'Title' : path.split('/')[-1],
            'Speech' : speech_file.read(),
            'Date' : date
        }   
    nm_speeches.append(speech)
nm_df = pd.DataFrame(nm_speeches)
nm_df.head(10)

,Date,Speech,Title
0,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",2017\421_03_Jan_2017_PMs_address_at_the_inaugu...
1,2017-01-05,"à¤¶à¥à¤°à¥ à¤ªà¤à¤¨à¤¾ à¤¸à¤¾à¤¹à¤¿à¤¬, à¤...",2017\422_05_Jan_2017_PMs_address_at_350th_Birt...
2,2017-01-07,"Your Excellency, Prime Minister Antonio Costa,...",2017\423_07_Jan_2017_Media_Statement_by_the_PM...
3,2017-01-08,"Excellencies and friends,\nTo begin with, let ...",2017\424_08_Jan_2017_PMs_address_at_the_inaugu...
4,2017-01-09,à¤ªà¥à¤¯à¤¾à¤°à¥ à¤­à¤¾à¤à¤¯à¥à¤ à¤à¤° à...,2017\425_09_Jan_2017_PMs_address_at_Bhumi_Puja...
5,2017-01-09,I am delighted to be here at Gift City to inau...,2017\426_09_Jan_2017_PMs_address_at_the_Inaugu...
6,2017-01-09,Chief Minister of Gujarat Shri Vijay Rupani ji...,2017\427_09_Jan_2017_PMs_address_at_inaugurati...
7,2017-01-11,"Your Excellency, President Uhuru Kenyatta,\nDi...",2017\429_11_Jan_2017_PMs_statement_to_media_du...
8,2017-01-12,"Most respected Morari Bapu Ji, President of th...",2017\430_12_Jan_2017_PMs_address_to_the_inaugu...
9,2017-01-12,मंचस्थ सभी गण्यमान्य अतिथि और मेरे प्यारे नौजव...,2017\431_12_Jan_2017_PMs_address_to_inaugural_...


In [59]:
nm_df['Year'],nm_df['Month'],nm_df['Day'] = nm_df.Date.dt.year, nm_df.Date.dt.month, nm_df.Date.dt.day

In [60]:
nm_df.head()

,Date,Speech,Title,Year,Month,Day
0,2017-01-03,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",2017\421_03_Jan_2017_PMs_address_at_the_inaugu...,2017,1,3
1,2017-01-05,"à¤¶à¥à¤°à¥ à¤ªà¤à¤¨à¤¾ à¤¸à¤¾à¤¹à¤¿à¤¬, à¤...",2017\422_05_Jan_2017_PMs_address_at_350th_Birt...,2017,1,5
2,2017-01-07,"Your Excellency, Prime Minister Antonio Costa,...",2017\423_07_Jan_2017_Media_Statement_by_the_PM...,2017,1,7
3,2017-01-08,"Excellencies and friends,\nTo begin with, let ...",2017\424_08_Jan_2017_PMs_address_at_the_inaugu...,2017,1,8
4,2017-01-09,à¤ªà¥à¤¯à¤¾à¤°à¥ à¤­à¤¾à¤à¤¯à¥à¤ à¤à¤° à...,2017\425_09_Jan_2017_PMs_address_at_Bhumi_Puja...,2017,1,9


In [61]:
nm_df = nm_df[["Year","Month", "Day", "Title", "Speech", "Date"]]

In [62]:
nm_df.head()

,Year,Month,Day,Title,Speech,Date
0,2017,1,3,2017\421_03_Jan_2017_PMs_address_at_the_inaugu...,"Governor of Andhra Pradesh, Shri E. S. L. Nara...",2017-01-03
1,2017,1,5,2017\422_05_Jan_2017_PMs_address_at_350th_Birt...,"à¤¶à¥à¤°à¥ à¤ªà¤à¤¨à¤¾ à¤¸à¤¾à¤¹à¤¿à¤¬, à¤...",2017-01-05
2,2017,1,7,2017\423_07_Jan_2017_Media_Statement_by_the_PM...,"Your Excellency, Prime Minister Antonio Costa,...",2017-01-07
3,2017,1,8,2017\424_08_Jan_2017_PMs_address_at_the_inaugu...,"Excellencies and friends,\nTo begin with, let ...",2017-01-08
4,2017,1,9,2017\425_09_Jan_2017_PMs_address_at_Bhumi_Puja...,à¤ªà¥à¤¯à¤¾à¤°à¥ à¤­à¤¾à¤à¤¯à¥à¤ à¤à¤° à...,2017-01-09


In [63]:
nm_df.to_pickle('speeches_df_2017.pkl')